In [1]:
import numpy as np
from matplotlib import pyplot as plt

import import_ipynb
import Network_Jammed as Network

importing Jupyter notebook from Network_Jammed.ipynb


In [2]:
def osc(net, train_length, num_osc, eps):
    T = train_length * num_osc
    train_type = True
    working_net = net.copy()
    nets = [working_net.copy()]
    Bs = []
    Gs = []
    for t in range(T):
        if t%train_length == 0:
            train_type = not(train_type)
        if train_type == True:
            bulk_net, B = nets[-1].bulk_modulus(eps)
            shear_net, G = nets[-1].shear_modulus(eps)
            Bs.append(B)
            Gs.append(G)
            cut_bond_ind = np.argmax(np.abs(bulk_net.stresses()))
            nets.append(nets[-1].prune(cut_bond_ind))
        if train_type == False:
            bulk_net, B = nets[-1].bulk_modulus(eps)
            shear_net, G = nets[-1].shear_modulus(eps)
            Bs.append(B)
            Gs.append(G)
            cut_bond_ind = np.argmax(np.abs(shear_net.stresses()))
            nets.append(nets[-1].prune(cut_bond_ind))
    bulk_net, B = nets[-1].bulk_modulus(eps)
    shear_net, G = nets[-1].shear_modulus(eps)
    Bs.append(B)
    Gs.append(G)
    
    return np.array(nets), np.array(Bs), np.array(Gs) 

In [3]:
def train(net, T, eps, train_type = -1):
    working_net = net.copy()
    nets = [working_net.copy()]
    Bs = []
    Gs = []
    for t in range(T):
        if train_type == -1:
            bulk_net, B = nets[-1].bulk_modulus(eps)
            shear_net, G = nets[-1].shear_modulus(eps)
            Bs.append(B)
            Gs.append(G)
            cut_bond_ind = np.argmax(np.abs(bulk_net.stresses()))
            nets.append(nets[-1].prune(cut_bond_ind))
        if train_type == 1:
            bulk_net, B = nets[-1].bulk_modulus(eps)
            shear_net, G = nets[-1].shear_modulus(eps)
            Bs.append(B)
            Gs.append(G)
            cut_bond_ind = np.argmax(np.abs(shear_net.stresses()))
            nets.append(nets[-1].prune(cut_bond_ind))
    bulk_net, B = nets[-1].bulk_modulus(eps)
    shear_net, G = nets[-1].shear_modulus(eps)
    Bs.append(B)
    Gs.append(G)
    
    return np.array(nets), np.array(Bs), np.array(Gs) 

In [5]:
def poisson_ratios(Gs, Bs):
    v = (1 - Gs/Bs) / (1 + Gs/Bs)
    return v